# Power curves

Based on [this](https://deliveroo.engineering/2018/12/07/monte-carlo-power-analysis.html) post from Deliveroo engineering blog.

In [3]:
import random

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import bernoulli, binom, mannwhitneyu, norm, uniform
import seaborn as sns
from statsmodels.stats.weightstats import ttest_ind

sns.set_style("whitegrid")
%config InlineBackend.figure_format ='retina'

## Walk-through

Prepare data

In [14]:
# Sample data would be actual data measured over a fixed period of time prior to our
# experiment. For illustration purposes here we have generated data from a normal
# distribution.
sample_mean = 21.50
sample_sd = 12.91
sample_data = norm.rvs(loc=sample_mean, scale=sample_sd, size=20000)
sample_data

array([25.46440322, 25.10002404, 31.1763618 , ..., 33.12020466,
       16.87635585, 28.2148879 ])

Specify experiment parameters

In [89]:
sample_sizes = range(250, 20000 + 1, 250)  # Sample sizes we will test over
alpha = 0.05  # Our fixed alpha
num_runs = 20  # The number of simulations we will run per sample size
relative_effect = 1.03   # MDES - could try multiple if unsure what to use
alternative = "two-sided"  # Is the alternative one-sided or two-sided

Run simulations

In [91]:
power_dist = np.empty((len(sample_sizes), 3))

for i, sample_size in enumerate(sample_sizes):

    # Control data is sample of pre-experiment data of specified size. To
    # Create treatment data, simply multiply by relative effect size.
    control_data = sample_data[0:sample_size]
    variant_data = control_data * relative_effect
    
    significance_t_results = []
    significance_u_results = []
    for _ in range(0, num_runs):
        
        # Create treatment assignment vector and collect data for
        # control and treatment units
        assignment = binom.rvs(1, 0.5, size=sample_size)
        control_sample = control_data[assignment == True]
        variant_sample = variant_data[assignment == False]

        # Use Welch's t-test, make no assumptions on tests for equal variances
        t_stat, p_value_welch, _ = ttest_ind(
            control_sample, variant_sample, alternative=alternative, usevar="unequal"
        )
        # Use Mann-Whitney U-test
        u_stat, p_value_mw = mannwhitneyu(
            control_sample, variant_sample, alternative=alternative
        )

        # Test for significance
        significance_t_results.append(p_value_welch <= alpha)
        significance_u_results.append(p_value_mw <= alpha)
    
    # The power is the number of times we have a significant result
    # as we are assuming the alternative hypothesis is true
    power_t = np.mean(significance_t_results)
    power_u = np.mean(significance_u_results)
    power_dist[i,] = [sample_size, power_t, power_u]

In [92]:
power_dist = pd.DataFrame(power_dist, columns=["sample_size", "tpower", "upower"])
power_dist = power_dist.melt(
    id_vars="sample_size",
    value_vars=["tpower", "upower"],
    var_name="test",
    value_name="power",
)
labels = {"tpower": "MC Simulation t-test", "upower": "MC Simulation MWW U-test"}
power_dist["test"].replace(labels, inplace=True)
power_dist.head()

,sample_size,test,power
0,250.0,MC Simulation t-test,0.05
1,500.0,MC Simulation t-test,0.05
2,750.0,MC Simulation t-test,0.05
3,1000.0,MC Simulation t-test,0.10
4,1250.0,MC Simulation t-test,0.10


Plot power distribution

In [93]:
dots = (
    alt.Chart(power_dist)
    .mark_point()
    .encode(
        x=alt.X("sample_size", axis=alt.Axis(title="Sample size")),
        y=alt.Y("power", axis=alt.Axis(title="Power")),
        color=alt.Color("test", legend=alt.Legend(title="")),
    )
)

hline = (
    alt.Chart(power_dist)
    .mark_rule()
    .encode(
        y=alt.Y("a:Q", axis=alt.Axis(title="")),
    )
    .transform_calculate(a="0.8")
)

(
    dots
    + dots.transform_loess("sample_size", "power", groupby=["test"]).mark_line()
    + hline
)

/Users/fabian.gunzinger/.pyenv/versions/3.10.8/envs/wow/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

### Program

In [116]:
def get_power_dist(
    data,
    sample_min,
    sample_max,
    stepsize,
    relative_effect=1.03,
    sims=20,
    alpha=0.05,
    alternative_t="two-sided",
    alternative_u="two-sided",
    tests=["t", "u"],
):
    """
    Calculates power for each sample size and each specified test.
    Code is adapted version of https://deliveroo.engineering/2018/12/07/monte-carlo-power-analysis.html

    Parameters
    -----------
    data: ArrayLike the historical data based on which power is being calculated (required)
    sample_min : integer (required)
        minimum sample size for which power is calculated
    sample_max : integer (required)
        maximum sample size for which power is calculated
    stepsize : integer (required)
        number of sample sized skipped between each calculation
    relative_effect : float (optional)
        relative effect size of variant, default is 3 percent
    sims : integer (optional)
        number of simulations per sample size, default is 20
    alpha : float (optional)
        probability of Type I error, default is 5 percent
    alternative_t : string (optional)
        type of t-test performed, either 'two-sided' (default), 'larger', or 'smaller'
    alternative_u : string (optional)
        type of Mann-Whitney-Wilkinson U-test performed, either 'two-sided' (default), 'more', or 'less'
    tests : list of strings (optional)
        test results plotted, either ['t', 'u'] (default), ['t'], or ['u']


    Returns
    --------
    dataframe : a dataframe containing sample size and associated power levels for each test
    """
    sample_sizes = range(sample_min, sample_max + 1, stepsize)

    power_dist = np.empty((len(sample_sizes), 3))
    for i in range(0, len(sample_sizes)):
        N = sample_sizes[i]
        control_data = data[0:N]
        variant_data = control_data * relative_effect

        significance_tresults = []
        significance_uresults = []
        for j in range(0, sims):
            # Randomly allocate the sample data to the control and variant
            rv = binom.rvs(1, 0.5, size=N)
            control_sample = control_data[rv == True]
            variant_sample = variant_data[rv == False]

            # Calculate Welch's t-test and Mann-Whitney U-test
            ttest_result = ttest_ind(
                control_sample,
                variant_sample,
                alternative=alternative_t,
                usevar="unequal",
            )
            utest_result = mannwhitneyu(
                control_sample, variant_sample, alternative=alternative_u
            )

            # Test for significance and calculate power
            significance_tresults.append(ttest_result[1] <= alpha)
            tpower = np.mean(significance_tresults)
            significance_uresults.append(utest_result[1] <= alpha)
            upower = np.mean(significance_uresults)

        # Store results for sample size i
        power_dist[i,] = [N, tpower, upower]

    # Convert to dataframe and keep results for selected tests
    power_dist = pd.DataFrame(power_dist, columns=["sample_size", "t", "u"])
    power_dist = power_dist.melt(
        id_vars="sample_size",
        value_vars=["t", "u"],
        var_name="test",
        value_name="power",
    )
    power_dist = power_dist[power_dist["test"].isin(tests)]
    labels = {"t": "MC Simulation t-test", "u": "MC Simulation MWW U-test"}
    power_dist["test"].replace(labels, inplace=True)

    return power_dist


def plot_power_graph(data, x="sample_size", y="power", color="test", hline_pos="0.8"):
    """
    Creates a standardised power graph

    Parameters
    ----------
    data : dataframe (required)
        name of dataframe
    x : string (optional)
        name of x coordinate (sample size), default is 'sample_size'
    y : string (optional)
        name of y coordinate (power), default is 'power'
    color : string (optional)
        name of variable for color encoding, default is 'test'
    hline_pos : str of float (optional)
        position of horizontal line to indicate target power level, default is '0.8'

    Returns
    -------
    Altair plot: A plot showing level of power for each sample size for each test.
    """
    dots = (
        alt.Chart(data)
        .mark_point()
        .encode(
            x=alt.X(x, axis=alt.Axis(title="Sample size")),
            y=alt.Y(y, axis=alt.Axis(title="Power")),
            color=alt.Color(color, legend=alt.Legend(title="")),
        )
    )

    loess = dots.transform_loess(x, y, groupby=[color]).mark_line()

    hline = (
        alt.Chart(data)
        .mark_rule(color="red")
        .encode(
            y=alt.Y("a:Q", axis=alt.Axis(title="")),
        )
        .transform_calculate(a=hline_pos)
    )

    return dots + loess + hline

In [119]:
sample_size = 20_000
step_size = 500
newdata = norm.rvs(loc=20, scale=12, size=sample_size)
data = get_power_dist(newdata, 100, sample_size, step_size, tests=['u'])
plot_power_graph(data)

/Users/fabian.gunzinger/.pyenv/versions/3.10.8/envs/wow/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)